In [ ]:
%pip install scikit-learn

In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def load_dataset(extract_func):
    data = []
    labels = []
    
    tumor_path = '../data/preprocessed_class/tumor/'
    healthy_path = '../data/preprocessed_class/pancreas/'
    
    for image_path in os.listdir(tumor_path):
        image = cv2.imread(os.path.join(tumor_path, image_path))
        if image is not None:
            descriptors = extract_func(image)
            if descriptors is not None:
                data.append(descriptors.flatten())
                labels.append(1)  # 1 for tumor
    
    for image_path in os.listdir(healthy_path):
        image = cv2.imread(os.path.join(healthy_path, image_path))
        if image is not None:
            descriptors = extract_func(image)
            if descriptors is not None:
                data.append(descriptors.flatten())
                labels.append(0)  # 0 for pancreas
    
    return np.array(data), np.array(labels)

In [ ]:
# Function to extract SIFT features from an image
def extract_sift_features(image, target_shape=(128,128)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=21, contrastThreshold=0.0, edgeThreshold=7, sigma=9.5)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros((128,128))
    return descriptors

In [ ]:
def extract_surf_features(image, target_shape=(128,64)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a SURF object with specified parameters
    surf = cv2.xfeatures2d.SURF_create(hessianThreshold=10, nOctaves=2, nOctaveLayers=4, extended=False, upright=True)
    
    # Detect and compute SURF keypoints and descriptors
    keypoints, descriptors = surf.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)

    return descriptors

In [ ]:
def extract_fast_features(image, target_shape=(32,32)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a FAST object with specified parameters
    fast = cv2.FastFeatureDetector_create(threshold=3, nonmaxSuppression=True, type=0)
    
    # Detect FAST keypoints
    keypoints = fast.detect(gray, None)
    
    # Compute descriptors (using ORB as an example, you can use any descriptor method)
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.compute(gray, keypoints)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)
    
    return descriptors

In [ ]:
def extract_orb_features(image, target_shape=(512,32)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create an ORB object with specified parameters
    orb = cv2.ORB_create(nfeatures=500, scaleFactor=1.03, nlevels=2, edgeThreshold=21, firstLevel=0, WTA_K=2, scoreType=0, patchSize=31, fastThreshold=0)
    
    # Detect ORB keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)

    return descriptors

In [ ]:
def extract_brisk_features(image, target_shape=(256,64)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a BRISK object with specified parameters
    brisk = cv2.BRISK_create(0, 1, 1.2)
    
    # Detect and compute BRISK keypoints and descriptors
    keypoints, descriptors = brisk.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)
    
    return descriptors

In [ ]:
def extract_kaze_features(image, target_shape=(128,128)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a KAZE object with specified parameters
    kaze = cv2.KAZE_create(extended=True, upright=False, threshold=0.0, nOctaves=4, nOctaveLayers=1, diffusivity=1)
    
    # Detect and compute KAZE keypoints and descriptors
    keypoints, descriptors = kaze.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)
      
    return descriptors

In [ ]:
def extract_akaze_features(image, target_shape=(128,16)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create an AKAZE object with specified parameters
    akaze = cv2.AKAZE_create(descriptor_type=5, descriptor_size=128, descriptor_channels=3, threshold=0.0, nOctaves=1, nOctaveLayers=1, diffusivity=0)
    
    # Detect and compute AKAZE keypoints and descriptors
    keypoints, descriptors = akaze.detectAndCompute(gray, None)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)
    
    return descriptors

In [ ]:
def extract_agast_features(image, target_shape=(32,32)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create an AGAST object with specified parameters
    agast = cv2.AgastFeatureDetector_create(threshold=7, nonmaxSuppression=False, type=0)
    
    # Detect AGAST keypoints
    keypoints = agast.detect(gray, None)
    
    # Compute descriptors using BRIEF
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    keypoints, descriptors = brief.compute(gray, keypoints)
    
    if descriptors is not None:
        if descriptors.shape[0] < target_shape[0]:
            # Zero-padding if descriptors are shorter
            descriptors = np.pad(descriptors, ((0, target_shape[0] - descriptors.shape[0]), (0, 0)), 'constant')
        elif descriptors.shape[0] > target_shape[0]:
            # Resize if descriptors are longer
            descriptors = cv2.resize(descriptors, (target_shape[1], target_shape[0]))
    else:
      descriptors = np.zeros(target_shape)
    
    return descriptors

In [ ]:
def perform(data, labels):
  print(f"dataset: {data.shape},")
  # Split the dataset into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

  # Create a Support Vector Machine (SVM) classifier
  classifier = SVC()

  # Define the parameter grid for grid search
  param_grid = {
      'C': [0.1, 1, 10, 100],
      'kernel': ['linear', 'rbf', 'poly'],
      'gamma': ['scale', 'auto'],
  }

  # Use GridSearchCV for hyperparameter tuning
  grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', verbose=1)
  grid_search.fit(X_train, y_train)

  # Get the best hyperparameters
  best_params = grid_search.best_params_
  print(f"Best hyperparameters: {best_params}")

  # Train the classifier on the entire training set with the best hyperparameters
  best_classifier = grid_search.best_estimator_
  best_classifier.fit(X_train, y_train)

  # Make predictions on the test set
  predictions = best_classifier.predict(X_test)

  # Evaluate the model
  accuracy = accuracy_score(y_test, predictions)
  print(f"Accuracy on the test set: {accuracy}")

In [ ]:
print("SIFT")
data, labels = load_dataset(extract_sift_features)
perform(data, labels)

In [ ]:
print("SURF")
data, labels = load_dataset(extract_surf_features)
perform(data, labels)

In [ ]:
print("FAST")
data, labels = load_dataset(extract_fast_features)
perform(data, labels)

In [ ]:
print("ORB")
data, labels = load_dataset(extract_orb_features)
perform(data, labels)

In [ ]:
print("BRISK")
data, labels = load_dataset(extract_brisk_features)
perform(data, labels)

In [ ]:
print("KAZE")
data, labels = load_dataset(extract_kaze_features)
perform(data, labels)

In [ ]:
print("AKAZE")
data, labels = load_dataset(extract_akaze_features)
perform(data, labels)

In [ ]:
print("AGAST")
data, labels = load_dataset(extract_agast_features)
perform(data, labels)